In [2]:
import numpy as np
import tensorflow as tf
import copy
from kerassurgeon.operations import delete_channels
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

In [26]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_train /= 255.
x_test = x_test.astype('float32')
x_test /= 255.

In [22]:
x_test.shape[1:4]

(28, 28, 1)

In [28]:
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=x_train.shape[1:4], activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(28, kernel_size=(3,3), input_shape=x_train.shape[1:4], activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(32, activation=tf.nn.relu))
model.add(Dense(10,activation=tf.nn.softmax))

In [31]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 20s 341us/step - loss: 0.1863 - accuracy: 0.9425
Epoch 2/10
60000/60000 [==============================] - 20s 341us/step - loss: 0.0585 - accuracy: 0.9816
Epoch 3/10
60000/60000 [==============================] - 21s 356us/step - loss: 0.0419 - accuracy: 0.9871
Epoch 4/10
60000/60000 [==============================] - 21s 358us/step - loss: 0.0318 - accuracy: 0.9904
Epoch 5/10
60000/60000 [==============================] - 22s 375us/step - loss: 0.0246 - accuracy: 0.9922
Epoch 6/10
60000/60000 [==============================] - 22s 359us/step - loss: 0.0207 - accuracy: 0.9932
Epoch 7/10
60000/60000 [==============================] - 20s 328us/step - loss: 0.0178 - accuracy: 0.9941
Epoch 8/10
60000/60000 [==============================] - 19s 315us/step - loss: 0.0136 - accuracy: 0.9954
Epoch 9/10
60000/60000 [==============================] - 19s 315us/step - loss: 0.0126 - accuracy: 0.9958
Epoch 10/10
60000/60000 [============

In [32]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 104us/step


[0.03287297285744397, 0.9904000163078308]

In [38]:
model.count_params()

51630

In [35]:
def scoring(model, layer_index):       #it returns normalized scores of layer 'layer_index'
    numberOfFilters = model.get_layer(index = layer_index).get_weights()[0].shape[3]
#     print(numberOfFilters)
    scores = []
    for i in range (numberOfFilters):
        scores.append(np.linalg.norm( model.get_layer(index = layer_index).get_weights()[0][:,:,:,i]) + np.linalg.norm( model.get_layer(index = layer_index).get_weights()[1][i])) 
    
    maxScore = max(scores)
    return np.array(scores)/maxScore

In [34]:
def pruning(model, threshold, epochs):
    iteration = 0
    while True:
        iteration +=1
        print("iteration #" + str(iteration))
        filterLayers = []                                
        for i in range(len(model.layers)):
            if str(model.layers[i]).count("convolutional") > 0:
                filterLayers.append(i)

        scoresOfLayers = {}
        for i in filterLayers:
            scoresOfLayers["l"+str(i)] = scoring(model, i)

        flagToProceed = False
        for i in scoresOfLayers:
            if scoresOfLayers[i].shape[0] > 1:
                flagToProceed = True

        if flagToProceed == False:
            print("All layers have only 1 filter left, stopping it !!!")
            break

        minimumScoresPerLayer = []
        for i in scoresOfLayers:
            minimumScoresPerLayer.append(scoresOfLayers[i].min())
        pruningLayer = filterLayers[np.array(minimumScoresPerLayer).argmin()]
        pruningFilter = scoresOfLayers["l" + str(pruningLayer)].argmin()

        print(scoresOfLayers)
        print("====================")
        print(pruningLayer, pruningFilter)
        print("====================")

        modelDeepCopied = copy.deepcopy(model)
        model_new = delete_channels(model, model.layers[pruningLayer], [pruningFilter])

        model_new.compile(optimizer='adam', 
                      loss='sparse_categorical_crossentropy', 
                      metrics=['accuracy'])
        model_new.fit(x=x_train,y=y_train, epochs=epochs)


        print("test accuracy:")
        print(model_new.evaluate(x_test, y_test)[1])

        if model_new.evaluate(x_test, y_test)[1] > threshold:
            model = model_new
        else:
            print("accuracy dropped below threhold, stopping it !!!")
            break


In [36]:
pruning(model, 0.96, 5)

iteration #1
{'l0': array([0.606814  , 0.9229351 , 0.7232515 , 0.36631894, 0.9145028 ,
       0.5678287 , 0.30092943, 0.7762076 , 0.45462617, 0.41574916,
       0.75885916, 0.5261007 , 0.8734554 , 0.718889  , 0.7583239 ,
       0.38806838, 1.        , 0.7461518 , 0.6514504 , 0.9481061 ,
       0.88325757, 0.4384668 , 0.4436274 , 0.9304094 , 0.44880658,
       0.93886197, 0.96065843, 0.9023028 ], dtype=float32), 'l2': array([1.        , 0.7934735 , 0.94585603, 0.8329287 , 0.82306945,
       0.74706537, 0.85043347, 0.94606376, 0.8607214 , 0.8293997 ,
       0.97740555, 0.7578261 , 0.7999112 , 0.85168225, 0.9665853 ,
       0.9070671 , 0.7775275 , 0.7413592 , 0.8106757 , 0.8239844 ,
       0.83023244, 0.8801974 , 0.9994074 , 0.8733474 , 0.7896383 ,
       0.90882677, 0.8787805 , 0.78733444], dtype=float32)}
0 6
Deleting 1/28 channels from layer: conv2d_11
Epoch 1/5
60000/60000 [==============================] - 19s 324us/step - loss: 0.0099 - accuracy: 0.9967
Epoch 2/5
60000/60000 [======

Deleting 1/23 channels from layer: conv2d_11
Epoch 1/5
60000/60000 [==============================] - 18s 303us/step - loss: 0.0038 - accuracy: 0.9989
Epoch 2/5
60000/60000 [==============================] - 18s 297us/step - loss: 0.0025 - accuracy: 0.9991
Epoch 3/5
60000/60000 [==============================] - 18s 299us/step - loss: 0.0029 - accuracy: 0.9990
Epoch 4/5
60000/60000 [==============================] - 18s 297us/step - loss: 0.0026 - accuracy: 0.9992
Epoch 5/5
60000/60000 [==============================] - 18s 302us/step - loss: 0.0035 - accuracy: 0.9989
test accuracy:
10000/10000 [==============================] - 1s 120us/step
0.9911999702453613
10000/10000 [==============================] - 1s 102us/step
iteration #7
{'l1': array([0.7363617 , 0.81606406, 0.570308  , 0.7879637 , 0.7132295 ,
       0.6954135 , 0.4791025 , 0.69660455, 0.5771819 , 0.7845995 ,
       0.7594074 , 0.87967855, 1.        , 0.7877985 , 0.5780875 ,
       0.83062553, 0.78096247, 0.9021224 , 0.553

Deleting 1/17 channels from layer: conv2d_11
Epoch 1/5
60000/60000 [==============================] - 16s 268us/step - loss: 0.0030 - accuracy: 0.9992
Epoch 2/5
60000/60000 [==============================] - 16s 261us/step - loss: 0.0021 - accuracy: 0.9995
Epoch 3/5
60000/60000 [==============================] - 16s 261us/step - loss: 0.0023 - accuracy: 0.9994
Epoch 4/5
60000/60000 [==============================] - 16s 262us/step - loss: 0.0012 - accuracy: 0.9997
Epoch 5/5
60000/60000 [==============================] - 16s 264us/step - loss: 0.0026 - accuracy: 0.9994
test accuracy:
10000/10000 [==============================] - 1s 130us/step
0.9909999966621399
10000/10000 [==============================] - 1s 101us/step
iteration #13
{'l1': array([0.86819273, 0.79950505, 0.66314363, 0.7459049 , 0.742827  ,
       0.79112417, 0.87211585, 0.9976168 , 1.        , 0.734746  ,
       0.7255802 , 0.70646095, 0.8468073 , 0.8199129 , 0.8366181 ,
       0.72825617], dtype=float32), 'l3': array

Deleting 1/11 channels from layer: conv2d_11
Epoch 1/5
60000/60000 [==============================] - 16s 262us/step - loss: 0.0026 - accuracy: 0.9991
Epoch 2/5
60000/60000 [==============================] - 15s 254us/step - loss: 0.0025 - accuracy: 0.9994
Epoch 3/5
60000/60000 [==============================] - 15s 253us/step - loss: 0.0012 - accuracy: 0.9996
Epoch 4/5
60000/60000 [==============================] - 15s 256us/step - loss: 0.0017 - accuracy: 0.9994
Epoch 5/5
60000/60000 [==============================] - 15s 256us/step - loss: 0.0014 - accuracy: 0.9996
test accuracy:
10000/10000 [==============================] - 2s 151us/step
0.9901999831199646
10000/10000 [==============================] - 1s 113us/step
iteration #19
{'l1': array([0.84410363, 0.77391195, 0.8651618 , 0.801484  , 1.        ,
       0.9665672 , 0.78947496, 0.802144  , 0.86259156, 0.85398227],
      dtype=float32), 'l3': array([0.96387154, 0.79109913, 0.8264254 , 0.85653436, 0.8774328 ,
       0.7612279 ,

60000/60000 [==============================] - 18s 296us/step - loss: 0.0032 - accuracy: 0.9990
Epoch 3/5
60000/60000 [==============================] - 18s 296us/step - loss: 0.0023 - accuracy: 0.9992
Epoch 4/5
60000/60000 [==============================] - 18s 296us/step - loss: 0.0029 - accuracy: 0.9991
Epoch 5/5
60000/60000 [==============================] - 18s 297us/step - loss: 0.0011 - accuracy: 0.9996
test accuracy:
10000/10000 [==============================] - 2s 193us/step
0.9872999787330627
10000/10000 [==============================] - 1s 145us/step
iteration #25
{'l1': array([0.8443758 , 0.80784196, 0.84463423, 1.        , 0.95604837,
       0.7648759 , 0.8068197 ], dtype=float32), 'l3': array([0.86784434, 0.80328053, 0.9424636 , 0.8624853 , 0.9011454 ,
       0.78698397, 0.9560017 , 0.8606216 , 0.8260157 , 1.        ,
       0.94602567, 0.9270304 , 0.7538585 , 0.79489774, 0.9910279 ,
       0.9064664 , 0.80176187, 0.93440706, 0.91998434, 0.8990109 ,
       0.93885446, 0

Deleting 1/21 channels from layer: conv2d_12
Epoch 1/5
60000/60000 [==============================] - 17s 290us/step - loss: 0.0033 - accuracy: 0.9988
Epoch 2/5
60000/60000 [==============================] - 17s 279us/step - loss: 0.0013 - accuracy: 0.9997
Epoch 3/5
60000/60000 [==============================] - 17s 279us/step - loss: 0.0027 - accuracy: 0.9992
Epoch 4/5
60000/60000 [==============================] - 17s 278us/step - loss: 0.0015 - accuracy: 0.9995
Epoch 5/5
60000/60000 [==============================] - 17s 279us/step - loss: 0.0019 - accuracy: 0.9995
test accuracy:
10000/10000 [==============================] - 2s 209us/step
0.9879000186920166
10000/10000 [==============================] - 2s 151us/step
iteration #32
{'l1': array([0.8158256 , 0.85256535, 1.        , 0.9592188 , 0.7904335 ],
      dtype=float32), 'l3': array([0.7795737 , 0.9105023 , 0.85891455, 0.8648841 , 0.7919098 ,
       0.9163253 , 0.86025804, 0.7938032 , 0.9375602 , 0.80888784,
       0.9196509 ,

60000/60000 [==============================] - 21s 353us/step - loss: 0.0033 - accuracy: 0.9989
Epoch 3/5
60000/60000 [==============================] - 21s 349us/step - loss: 0.0031 - accuracy: 0.9990
Epoch 4/5
60000/60000 [==============================] - 21s 355us/step - loss: 0.0031 - accuracy: 0.9990
Epoch 5/5
60000/60000 [==============================] - 21s 348us/step - loss: 0.0032 - accuracy: 0.9990
test accuracy:
10000/10000 [==============================] - 3s 255us/step
0.9860000014305115
10000/10000 [==============================] - 2s 164us/step
iteration #39
{'l1': array([0.8914092 , 1.        , 0.93888414], dtype=float32), 'l3': array([0.92651707, 0.9022754 , 0.95650864, 0.7871371 , 0.82855946,
       0.9638234 , 0.9077628 , 0.89921093, 1.        , 0.85654294,
       0.8639742 , 0.867986  , 0.8448034 , 0.97076166, 0.7949864 ],
      dtype=float32)}
3 3
Deleting 1/15 channels from layer: conv2d_12
Epoch 1/5
60000/60000 [==============================] - 22s 366us/ste

60000/60000 [==============================] - 19s 323us/step - loss: 0.2596 - accuracy: 0.9468
Epoch 2/5
60000/60000 [==============================] - 18s 307us/step - loss: 0.0754 - accuracy: 0.9775
Epoch 3/5
60000/60000 [==============================] - 18s 306us/step - loss: 0.0538 - accuracy: 0.9827
Epoch 4/5
60000/60000 [==============================] - 18s 307us/step - loss: 0.0426 - accuracy: 0.9860
Epoch 5/5
60000/60000 [==============================] - 18s 307us/step - loss: 0.0358 - accuracy: 0.9884
test accuracy:
10000/10000 [==============================] - 3s 260us/step
0.9797000288963318
10000/10000 [==============================] - 2s 179us/step
iteration #47
{'l1': array([1.        , 0.71729106], dtype=float32), 'l3': array([0.8339063 , 1.        , 0.8350115 , 0.793896  , 0.97200465,
       0.9037127 , 0.98189116, 0.87107646], dtype=float32)}
1 1
Deleting 1/2 channels from layer: conv2d_11
Epoch 1/5
60000/60000 [==============================] - 19s 324us/step - 

In [ ]:
modelDeepCopied          
# final PRUNED model.
# count_params = 3528,
# test accuracy = 0.9663000106811523
# inference time = 5s 345us